<a href="https://colab.research.google.com/github/sheelasapkota59/CNN_Model/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy

In [ ]:
!pip install tensorflow

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

In [ ]:
#load preitrained ResNet50 as feature extractor
# The final output will be a 1D vector representing the high-level features of the image extracted
# by the ResNet50 model, which you can use for tasks like feature comparison, similarity calculation,
# or as input to another model.

base_model = ResNet50(weights = "imagenet" , include_top = False , pooling = "avg")

# feature extraction
def extract_feature(image_path_past, image_path_present):
  """extract feature from image using resnet50"""
  present_img = cv2.imread(image_path_present) # Load image
  if not os.path.exists(image_path_present):
        raise ValueError(f"Image not found: {image_path_present}")

  past_img = cv2.imread(image_path_past )
  if not os.path.exists(image_path_past):
        raise ValueError(f"Image not found: {image_path_past}")


  present_img = cv2.resize(present_img , (224 , 224)) # Resize present img to match ResNet50 input size
  past_img = cv2.resize(past_img , (224, 224))  # Resize past img to match ResNet50 input size

  present_img = preprocess_input(present_img)  # Preprocess image for ResNet50 (scaling pixel values)
  past_img = preprocess_input(past_img)

  present_img = np.expand_dims(present_img, axis = 0) # Add batch dimension
  past_img = np.expand_dims(past_img, axis = 0)

 # Combine the two images into a batch
  batch = np.concatenate((past_img , present_img) , axis =0) # Shape becomes (2, 224, 224, 3)

  features_batch = base_model.predict(batch)  # Get features from ResNet50

  # Return features for both images as flattened vectors(1D vector)
  past_features = features_batch[0].flatten()
  present_features = features_batch[1].flatten()

  return past_features , present_features    #define siamese network



In [ ]:
def build_siamese_network():
  input_shape = (2048,)

  #input layers for two images
  present_A = Input(shape = input_shape)
  past_A = Input(shape = input_shape)

#calculate absolute differences between feature vectors
  l1_layer = Lambda(lambda tensors : tf.abs(tensors[0] - tensors[1]))
  l1_distance = l1_layer([present_A , past_A])

  #final classification layer
  output = Dense(1 , activation=  "sigmoid")(l1_distance) # Sigmoid for similarity (0 or 1)

  #define siamese model
  model = Model(inputs=[present_A , past_A] , outputs=output)
  model.compile(loss="binary_crossentropy" , optimizer= "adam" , metrics=["accuracy"])

  return model



In [ ]:
data_folder_old = "/content/CNN/road1.png"
data_folder_new = "/content/CNN/road2.png"

past_img = [data_folder_old]
present_img = [data_folder_new]

# # Now, you can proceed with loading the images using these paths
# f1 = extract_feature(past_img)
# f2 = extract_feature(present_img)


x1 , x2 , y = [] , [] ,[]

#prepare dataset for training
for old_img in past_img:
  for new_img in  present_img:
    f1 , f2  = extract_feature(old_img , new_img)
    #  f2 = extract_feature(new_img)
    similarity_score = np.linalg.norm(f1-f2)  # Euclidean Distance

    x1.append(f1)
    x2.append(f2)
    y.append(1 if similarity_score < 100 else 0) # Label as similar if below threshold


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [ ]:
#convert to numpy arrays
x1 , x2, y = np.array(x1) , np.array(x2) , np.array(y)

#build and train siamese model
siamese_model = build_siamese_network()
siamese_model.fit([x1 , x2] , y , epochs = 18 , batch_size = 2 )

#evaluation model
y_pred = siamese_model.predict([x1 , x2])
y_pred = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y , y_pred)
precision = precision_score(y , y_pred)
recall = recall_score (y , y_pred)
f1 = f1_score(y , y_pred)

print(f"model evaluation : Accuracy= {accuracy:.2f}, Precision={precision: .2f},Recall={recall:.2f}, F1-score={f1:.2f}")


Epoch 1/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 0.7427
Epoch 2/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 1.0000 - loss: 0.5903
Epoch 3/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - accuracy: 1.0000 - loss: 0.4630
Epoch 4/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 1.0000 - loss: 0.3597
Epoch 5/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - accuracy: 1.0000 - loss: 0.2779
Epoch 6/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 1.0000 - loss: 0.2145
Epoch 7/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 1.0000 - loss: 0.1660
Epoch 8/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 1.0000 - loss: 0.1293
Epoch 9/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 1.0000 - loss: 0.1015
Epoch 10/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 1.0000 - loss: 0.0805
Epoch 11/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 0.0646
Epoch 12/18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 1.0000 - loss

In [ ]:
#auto label assignment
for i , new_imgs in enumerate(present_img):
  for old_imgs in past_img:
    f1, f2 = extract_feature(new_img, old_img)

    # Reshape f1 and f2 to add batch dimension
    f1 = np.expand_dims(f1, axis=0)
    f2 = np.expand_dims(f2, axis=0)

    prediction = siamese_model.predict([[f1] , [f2]])[0][0]

    if prediction > 0.85:   # If high similarity, assign label
      print(f"Label Assignment: {os.path.basename(new_imgs)} -> {os.path.basename(old_imgs)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_895', 'keras_tensor_896']
Received: inputs=(('Tensor(shape=(1, 2048))',), ('Tensor(shape=(1, 2048))',))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Label Assignment: road2.png -> road1.png
